In [1]:
import torch
import gurobipy as gp

In [2]:
from learn.info import ModelInfo
from learn.feature import VarFeature, ConFeature, EdgFeature
from learn.train import Inst

In [3]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
np.random.seed(0)

def maximum_independent_set_problem(
    num_nodes=20,
    edge_prob=0.3,
    print_output=True
):
    edges = []
    for i in range(num_nodes):
        for j in range(i+1, num_nodes):
            if np.random.rand() < edge_prob:
                edges.append((i, j))
    m = gp.Model("maximum_independent_set")
    x = m.addVars(num_nodes, vtype=GRB.BINARY, name="x")
    for (i, j) in edges:
        m.addConstr(x[i] + x[j] <= 1, name=f"edge_{i}_{j}")
    m.setObjective(gp.quicksum(x[i] for i in range(num_nodes)), GRB.MAXIMIZE)
    m.update()
    return m

In [4]:
var_features = []
con_features = []
edg_features = []
solutions = []

for i in range(100):
    m = maximum_independent_set_problem()
    info = ModelInfo.from_model(m)
    var_features.append(VarFeature.from_info(info.var_info, info.obj_info))
    con_features.append(ConFeature.from_info(info.con_info))
    edg_features.append(EdgFeature.from_info(info.con_info))
    m.optimize()
    s = [v.x for v in m.getVars()]
    solutions.append(s)

inst = Inst(var_features, con_features, edg_features, solutions)

Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 22.4.0 22E252)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 56 rows, 20 columns and 112 nonzeros
Model fingerprint: 0xeed75126
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 35 rows and 5 columns
Presolve time: 0.00s
Presolved: 21 rows, 15 columns, 52 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)

Root relaxation: objective 7.333333e+00, 13 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
c_v_edges, v_c_edges, node_features, edge_features, n_var, n_con = inst.xs
ys = inst.ys

In [6]:
import torch
from learn.model import FocalLoss, SpGAT
import torch.optim as optim

model = SpGAT(
    nfeat=inst.xs[2][0].shape[1],
    nhid=64,
    nclass=2,
    dropout=0.5,
    nheads=6,
    alpha=0.2
)

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-3)

In [ ]:
import random

for epoch in range(500):
    
    model.train()
    optimizer.zero_grad()
    
    agg_loss = 0
    inst_idxs = list(range(len(ys)))
    random.shuffle(inst_idxs)
    
    for i in inst_idxs:
        
        train_idx = torch.as_tensor(range(n_var[i]), dtype=torch.int32)
        output, edge_features[i] = model(
            node_features[i], 
            c_v_edges[i], 
            v_c_edges[i], 
            edge_features[i].detach()
        )
        fl = FocalLoss()
        loss = fl(output[train_idx], ys[i][train_idx])
        agg_loss += loss
        
    print(output[train_idx].detach().numpy(), ys[i][train_idx].detach().numpy())
    print(agg_loss)
    print('-'*100)
    
    agg_loss.backward()
    optimizer.step()

[[0.4945595  0.5054405 ]
 [0.501283   0.49871704]
 [0.4856809  0.5143191 ]
 [0.50044435 0.4995557 ]
 [0.493274   0.50672597]
 [0.48383883 0.5161611 ]
 [0.49614304 0.5038569 ]
 [0.4886638  0.5113362 ]
 [0.5004187  0.49958128]
 [0.48001343 0.51998657]
 [0.49985835 0.5001417 ]
 [0.49953648 0.50046355]
 [0.49536106 0.50463885]
 [0.50020397 0.499796  ]
 [0.50300086 0.4969991 ]
 [0.48803356 0.5119665 ]
 [0.49367982 0.5063202 ]
 [0.4971363  0.50286376]
 [0.5000864  0.4999136 ]
 [0.49202397 0.507976  ]] [0 0 0 1 1 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1]
tensor(17.3972, grad_fn=<AddBackward0>)
----------------------------------------------------------------------------------------------------
[[0.8461225  0.15387753]
 [0.85087603 0.14912401]
 [0.8464635  0.15353653]
 [0.85038054 0.14961948]
 [0.8491668  0.15083317]
 [0.8503813  0.14961863]
 [0.84810156 0.15189847]
 [0.84787893 0.1521211 ]
 [0.85132176 0.14867818]
 [0.84892    0.15108006]
 [0.8487695  0.15123053]
 [0.8499551  0.1500449 ]
 [0.8492521  0.1

In [ ]:
assert 1 == 2

In [ ]:
inst = Inst(
    [VarFeature.from_info(info.var_info, info.obj_info)],
    [ConFeature.from_info(info.con_info)],
    [EdgFeature.from_info(info.con_info)],
    [[v.x for v in m.getVars()]]
)

In [ ]:
import random

for epoch in range(10):
    
    model.train()
    optimizer.zero_grad()
    
    agg_loss = 0
    for i in range(5):
        inst_idx = random.randint(0, len(ys) - 1)
        train_idx = torch.as_tensor(range(2), dtype=torch.int32)
        
        output, edge_features[inst_idx] = model(
            node_features[inst_idx], 
            c_v_edges[inst_idx], 
            v_c_edges[inst_idx], 
            edge_features[inst_idx].detach()
        )
        fl = FocalLoss()
        loss = fl(output[train_idx], ys[inst_idx][train_idx])
        agg_loss += loss
        
    print(output[train_idx].detach().numpy(), ys[inst_idx][train_idx].detach().numpy())
    print(agg_loss)
    
    agg_loss.backward()
    optimizer.step()

In [ ]:
output

In [ ]:
train_idx

In [ ]:
output

In [ ]:
assert 1 == 2

In [ ]:
def train(epoch, num):
    global data_edge_features
    t = time.time()

    output, data_edge_features[num] = model(data_features[num], data_edge_A[num], data_edge_B[num], data_edge_features[num].detach())
    print(data_solution[num][idx_train])

    lf = Focal_Loss(torch.as_tensor(data_labels[num]))
    loss_train = lf(output[idx_train], data_solution[num][idx_train])

    return loss_train

In [ ]:
t_total = time.time()
loss_values = []
for epoch in range(args.epochs):
    model.train()
    optimizer.zero_grad()
    now_loss = 0
    
    for i in range(5):
        now_data = random.randint(0, data_num - 1)
        now_loss += train(epoch, now_data)
        
    loss_values.append(now_loss)
    now_loss.backward()
    optimizer.step()
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(now_loss))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1

In [ ]:
# Model and optimizer
model = SpGAT(nfeat=data_features[0].shape[1],    # Feature dimension
            nhid=args.hidden,             # Feature dimension of each hidden layer
            nclass=int(data_solution[0].max()) + 1, # Number of classes
            dropout=args.dropout,         # Dropout
            nheads=args.nb_heads,         # Number of heads
            alpha=args.alpha)             # LeakyReLU alpha coefficient

optimizer = optim.Adam(model.parameters(),    
                       lr=args.lr,                        # Learning rate
                       weight_decay=args.weight_decay)    # Weight decay to prevent overfitting

if args.cuda: # Move to GPU
    model.to(device)
    for now_data in range(data_num):
        data_features[now_data] = data_features[now_data].to(device)
        data_labels[now_data] = data_labels[now_data].to(device)
        data_solution[now_data] = data_solution[now_data].to(device)
        data_edge_A[now_data] = data_edge_A[now_data].to(device)
        data_edge_B[now_data] = data_edge_B[now_data].to(device)
        data_edge_features[now_data] = data_edge_features[now_data].to(device)
        data_idx_train[now_data] = data_idx_train[now_data].to(device)


for now_data in range(data_num):
    data_features[now_data] = Variable(data_features[now_data])
    data_edge_A[now_data] = Variable(data_edge_A[now_data])
    data_edge_B[now_data] = Variable(data_edge_B[now_data])
    data_solution[now_data] = Variable(data_solution[now_data])
    # Define computation graph for automatic differentiation

def train(epoch, num):
    global data_edge_features
    t = time.time()

    output, data_edge_features[num] = model(data_features[num], data_edge_A[num], data_edge_B[num], data_edge_features[num].detach())
    print(data_solution[num][idx_train])

    lf = Focal_Loss(torch.as_tensor(data_labels[num]))
    loss_train = lf(output[idx_train], data_solution[num][idx_train])

    return loss_train

t_total = time.time()
loss_values = []
bad_counter = 0
best = args.epochs + 1
best_epoch = 0
for epoch in range(args.epochs):
    model.train()
    optimizer.zero_grad()
    now_loss = 0
    for i in range(5):
        now_data = random.randint(0, data_num - 1)
        now_loss += train(epoch, now_data)
    loss_values.append(now_loss)
    now_loss.backward()
    optimizer.step()
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(now_loss))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1